<a href="https://colab.research.google.com/github/YasirHabib/Deep-Learning-Advanced-Computer-Vision/blob/master/conv_block_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

In [0]:
import tensorflow as tf

In [0]:
def init_filter(shape):
  w = np.random.randn(*shape) * np.sqrt(2.0 / np.prod(shape[:-1]))
  return w.astype(np.float32)

In [0]:
class ConvLayer():
  def __init__(self, num_feature_maps, num_color_channels, filter_width, filter_height, stride, padding):
    sz = (filter_width, filter_height, num_color_channels, num_feature_maps)
    W = init_filter(sz)
    b = np.zeros(num_feature_maps, dtype=np.float32)

    self.W = tf.Variable(W)
    self.b = tf.Variable(b)
    self.stride = stride
    self.padding = padding
    
  def forward(self, X):
    conv_out = tf.nn.conv2d(X, self.W, strides=[1, self.stride, self.stride, 1], padding=self.padding)
    conv_out = tf.nn.bias_add(conv_out, self.b)
    return conv_out
  
  def copyFromKerasLayers(self, layer):
    # only 1 layer to copy from
    W, b = layer.get_weights()
    op1 = self.W.assign(W)
    op2 = self.b.assign(b)
    self.session.run((op1, op2))

  #def copyFromWeights(self, W, b):
  # op1 = self.W.assign(W)
  # op2 = self.b.assign(b)
  # self.session.run((op1, op2))

  def get_params(self):
    return [self.W, self.b]

In [0]:
class BatchNormLayer():
  def __init__(self, D):
    self.running_mean = tf.Variable(np.zeros(D, dtype=np.float32), trainable=False)
    self.running_var  = tf.Variable(np.ones(D, dtype=np.float32), trainable=False)
    self.gamma        = tf.Variable(np.ones(D, dtype=np.float32))
    self.beta         = tf.Variable(np.zeros(D, dtype=np.float32))

  def forward(self, X):
    return tf.nn.batch_normalization(X, self.running_mean, self.running_var, self.beta, self.gamma, 1e-3)
    
  def copyFromKerasLayers(self, layer):
    # only 1 layer to copy from
    # order:
    # gamma, beta, moving mean, moving variance
    gamma, beta, running_mean, running_var = layer.get_weights()
    op1 = self.running_mean.assign(running_mean)
    op2 = self.running_var.assign(running_var)
    op3 = self.gamma.assign(gamma)
    op4 = self.beta.assign(beta)
    self.session.run((op1, op2, op3, op4))

  def get_params(self):
    return [self.running_mean, self.running_var, self.gamma, self.beta]

In [0]:
class ConvBlock():
  def __init__(self, num_color_channels, fm_sizes, activation=tf.nn.relu):
    assert(len(fm_sizes) == 3)
    
    self.session = None
    self.activation = activation
    
    self.conv1 = ConvLayer(fm_sizes[0], num_color_channels, 1, 1, stride=1, padding='VALID')     # DO NOT USE SAME MODE WHEN YOU HAVE 1x1 Convolutions
    self.bn1 = BatchNormLayer(fm_sizes[0])
    self.conv2 = ConvLayer(fm_sizes[1], fm_sizes[0], 3, 3, stride=1, padding='SAME')
    self.bn2 = BatchNormLayer(fm_sizes[1])
    self.conv3 = ConvLayer(fm_sizes[2], fm_sizes[1], 1, 1, stride=1, padding='VALID')
    self.bn3 = BatchNormLayer(fm_sizes[2])
    
    # init shortcut branch
    # note: # feature maps shortcut = # feauture maps conv 3
    self.convs = ConvLayer(fm_sizes[2], num_color_channels, 1, 1, stride=1, padding='VALID')
    self.bns = BatchNormLayer(fm_sizes[2])
    
    # in case needed later
    self.layers = [
        self.conv1, self.bn1,
        self.conv2, self.bn2,
        self.conv3, self.bn3,
        self.convs, self.bns
    ]
    
    # this will not be used when input passed in from a previous layer
    self.input_ = tf.placeholder(tf.float32, shape=(1, 224, 224, num_color_channels))
    
    self.output = self.forward(self.input_)
    
  def forward(self, X):
    # main branch
    FX=self.conv1.forward(X)
    FX=self.bn1.forward(FX)
    FX=self.activation(FX)
    
    FX=self.conv2.forward(FX)
    FX=self.bn2.forward(FX)
    FX=self.activation(FX)
    
    FX=self.conv3.forward(FX)
    FX=self.bn3.forward(FX)
    
    # shortcut branch
    SX=self.convs.forward(X)
    SX=self.bns.forward(SX)
    
    return self.activation(FX+SX)
  
  def predict(self, X):
    assert(self.session is not None)
    return self.session.run(self.output, feed_dict={self.input_: X})
  
  def set_session(self, session):
    # need to make this a session
    # so assignment happens on sublayers too
    self.session = session
    self.conv1.session = session
    self.bn1.session = session
    self.conv2.session = session
    self.bn2.session = session
    self.conv3.session = session
    self.bn3.session = session
    self.convs.session = session
    self.bns.session = session
    
  def copyFromKerasLayers(self, layers):
    # [<keras.layers.convolutional.Conv2D at 0x117bd1978>,
    #  <keras.layers.normalization.BatchNormalization at 0x117bf84a8>,
    #  <keras.layers.core.Activation at 0x117c15fd0>,
    #  <keras.layers.convolutional.Conv2D at 0x117c23be0>,
    #  <keras.layers.normalization.BatchNormalization at 0x117c51978>,
    #  <keras.layers.core.Activation at 0x117c93518>,
    #  <keras.layers.convolutional.Conv2D at 0x117cc1518>,
    #  <keras.layers.convolutional.Conv2D at 0x117d21630>,
    #  <keras.layers.normalization.BatchNormalization at 0x117cd2a58>,
    #  <keras.layers.normalization.BatchNormalization at 0x117d44b00>,
    #  <keras.layers.merge.Add at 0x117dae748>,
    #  <keras.layers.core.Activation at 0x117da2eb8>]
    self.conv1.copyFromKerasLayers(layers[0])
    self.bn1.copyFromKerasLayers(layers[1])
    self.conv2.copyFromKerasLayers(layers[3])
    self.bn2.copyFromKerasLayers(layers[4])
    self.conv3.copyFromKerasLayers(layers[6])
    self.bn3.copyFromKerasLayers(layers[8])
    self.convs.copyFromKerasLayers(layers[7])
    self.bns.copyFromKerasLayers(layers[9])
    
  def get_params(self):
    params = []
    for layer in self.layers:
      params += layer.get_params()
    return params

In [7]:
if __name__ == '__main__':
  conv_block = ConvBlock(num_color_channels=3, fm_sizes=[64, 64, 256])
  
  # make a fake image
  X = np.random.random((1, 224, 224, 3))
  
  init = tf.global_variables_initializer()
  with tf.Session() as session:
    conv_block.set_session(session)
    session.run(init)
    
    output = conv_block.predict(X)
    print("output.shape:", output.shape)

output.shape: (1, 224, 224, 256)
